# Finding the Walking Duration of each Restaurant from the nearest Station Exit
Notebook used to extract walking distance between restaurant and nearest train station exit

In [1]:
import pandas as pd
import numpy as np
import re

import geopandas as gpd #conda install geopandas
# conda install -c conda-forge nbformat

import plotly.express as px #conda install -c plotly plotly_express
from plotly.express import choropleth_mapbox    
from shapely.geometry import shape, Point
import openrouteservice

import warnings
warnings.filterwarnings("ignore")

## 1. Basic Setup, Loading Datasets

In [2]:
# ---------------- Loading in Restaurant dataset ---------------- #
df_restaurant = pd.read_csv("data/full_restaurant_rating_data.csv")

# ---------------- Converting Dataset to GeoDataFrame ---------------- #
gdf_restaurant = gpd.GeoDataFrame(df_restaurant, geometry= gpd.points_from_xy(df_restaurant['longitude'], df_restaurant['latitude']), crs='epsg:4326')

In [3]:
# ---------------- Loading in Train Station Exit Locations ---------------- #
### Dataset does not have the train station name, so need to use Train Station Locations dataset to identify
gdf_staion_exit = gpd.read_file('data/train_station_exit.geojson', crs='epsg:4326')

# ---------------- Loading in Train Station Locations ---------------- #
df_station = pd.read_csv('data/train_station_location.csv')
gdf_station = gpd.GeoDataFrame(df_station, geometry= gpd.points_from_xy(df_station['Longitude'], df_station['Latitude']), crs='epsg:4326')

In [4]:
gdf_staion_exit

,Name,Description,geometry
0,kml_1,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.87094 1.33851 0.00000)
1,kml_2,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.87051 1.33858 0.00000)
2,kml_3,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.86190 1.31924 0.00000)
3,kml_4,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.86870 1.33107 0.00000)
4,kml_5,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.86933 1.33115 0.00000)
...,...,...,...
469,kml_470,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.88114 1.39828 0.00000)
470,kml_471,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.88114 1.39816 0.00000)
471,kml_472,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.88941 1.39720 0.00000)
472,kml_473,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.88912 1.39713 0.00000)


In [5]:
gdf_station

,OBJECTID,STN_NAME,STN_NO,X,Y,Latitude,Longitude,COLOR,geometry
0,12,ADMIRALTY MRT STATION,NS10,24402.1063,46918.1131,1.440585,103.800998,RED,POINT (103.80100 1.44059)
1,16,ALJUNIED MRT STATION,EW9,33518.6049,33190.0020,1.316433,103.882893,GREEN,POINT (103.88289 1.31643)
2,33,ANG MO KIO MRT STATION,NS16,29807.2655,39105.7720,1.369933,103.849553,RED,POINT (103.84955 1.36993)
3,81,BAKAU LRT STATION,SE3,36026.0821,41113.8766,1.388093,103.905418,OTHERS,POINT (103.90542 1.38809)
4,80,BANGKIT LRT STATION,BP9,21248.2460,40220.9693,1.380018,103.772667,OTHERS,POINT (103.77267 1.38002)
...,...,...,...,...,...,...,...,...,...
184,175,WOODLANDS SOUTH MRT STATION,TE3,23607.8309,45444.7113,1.427260,103.793863,OTHERS,POINT (103.79386 1.42726)
185,146,WOODLEIGH MRT STATION,NE11,32173.3186,35706.3794,1.339190,103.870808,PURPLE,POINT (103.87081 1.33919)
186,6,YEW TEE MRT STATION,NS5,18438.9791,42158.0124,1.397535,103.747431,RED,POINT (103.74743 1.39754)
187,41,YIO CHU KANG MRT STATION,NS15,29294.1283,40413.0820,1.381756,103.844944,RED,POINT (103.84494 1.38176)


## 2. Assign Train station of each station exits

In [ ]:
gdf_staion_exit['key'] = 1
gdf_station['key'] = 1

# dataframe where each station exit is mapped to each station
merged_station_exit = pd.merge(gdf_staion_exit, gdf_station, on='key').drop('key', 1)

# GeoSeries of all the Coordinates of Station Exits
geoseries_exit = gpd.GeoSeries(merged_station_exit['geometry_x'], crs='EPSG:4326').to_crs('EPSG:24500')
# GeoSeries of all the Coordinates of Station
geoseries_station = gpd.GeoSeries(merged_station_exit['geometry_y'], crs='EPSG:4326').to_crs('EPSG:24500')

# Computing Distance (in meters) of each exit to each station
merged_station_exit['distance'] = geoseries_exit.distance(geoseries_station)

# Obtaining the nearest station for each exit
df_nearest_station_to_each_exit = merged_station_exit.sort_values(['Name', 'distance'], ascending=True).groupby('Name').head(1)

# Dropping unnecessary columns
df_nearest_station_to_each_exit.drop(columns=['Description', 'X', 'Y', 'Latitude', 'Longitude', 'geometry_y'], inplace=True)
df_nearest_station_to_each_exit.rename(columns={'geometry_x': 'geometry'}, inplace=True)

# Converting to GeoDataFrame
df_nearest_station_to_each_exit['geometry'] = gpd.GeoSeries(df_nearest_station_to_each_exit['geometry'], crs='EPSG:4326')
gdf_nearest_station_to_each_exit = gpd.GeoDataFrame(df_nearest_station_to_each_exit, geometry= df_nearest_station_to_each_exit.geometry, crs='epsg:4326')

## 3. Find nearest train station exit for each restaurant

In [ ]:
gdf_nearest_station_to_each_exit['key'] = 1
gdf_restaurant['key'] = 1

# dataframe where each station exit is mapped to each station
merged_exit_restaurant = pd.merge(gdf_nearest_station_to_each_exit, gdf_restaurant, on='key').drop('key', 1)

# GeoSeries of all the Coordinates of Station Exits
geoseries_exit = gpd.GeoSeries(merged_exit_restaurant['geometry_x'], crs='EPSG:4326').to_crs('EPSG:24500')
# GeoSeries of all the Coordinates of Restaurants
geoseries_restaurant = gpd.GeoSeries(merged_exit_restaurant['geometry_y'], crs='EPSG:4326').to_crs('EPSG:24500')

# Computing Distance (in meters) of each restaurant to each exit
merged_exit_restaurant['distance'] = geoseries_exit.distance(geoseries_restaurant)

# Obtaining the nearest exit for each restaurant
df_restaurant_nearest_exit = merged_exit_restaurant.sort_values(['url', 'distance'], ascending=True).groupby('url').head(1)

# Dropping unnecessary columns
df_restaurant_nearest_exit = df_restaurant_nearest_exit[['Name', 'geometry_x', 'STN_NAME', 'STN_NO', 
                    'COLOR', 'distance', 'url', 'geometry_y']].copy()

df_restaurant_nearest_exit.rename(columns={
    'Name': 'exit_name',
    'STN_NAME': 'station_name',
    'STN_NO': 'station_no',
    'COLOR': 'station_line_color',
    'geometry_x': 'exit_geometry',
    'geometry_y': 'restaurant_geometry'
}, inplace=True)

# Converting to GeoDataFrame
df_restaurant_nearest_exit.reset_index(drop=True, inplace=True)

In [ ]:
# df_restaurant_nearest_exit.to_csv('data/restaurant_nearest_station_exit.csv', index=False, encoding='utf-8-sig')

In [216]:
df_restaurant_nearest_exit

,exit_name,exit_geometry,station_name,station_no,station_line_color,distance,url,restaurant_geometry
0,kml_9,POINT Z (103.851327253152 1.29685892134698 0),BRAS BASAH MRT STATION,CC2,YELLOW,167.415531,https://www.tripadvisor.com.sg/Restaurant_Revi...,POINT (103.85186 1.295443)
1,kml_273,POINT Z (103.802355553546 1.27210695832699 0),LABRADOR PARK MRT STATION,CC27,YELLOW,323.387371,https://www.tripadvisor.com.sg/Restaurant_Revi...,POINT (103.80167 1.274949)
2,kml_310,POINT Z (103.86307400627 1.31385250565917 0),BENDEMEER MRT STATION,DT23,BLUE,369.324140,https://www.tripadvisor.com.sg/Restaurant_Revi...,POINT (103.85977 1.31354)
3,kml_353,POINT Z (103.816994458984 1.28937070497306 0),REDHILL MRT STATION,EW18,GREEN,664.868461,https://www.tripadvisor.com.sg/Restaurant_Revi...,POINT (103.81824 1.28349)
4,kml_228,POINT Z (103.844389333915 1.32109486825779 0),NOVENA MRT STATION,NS20,RED,57.724200,https://www.tripadvisor.com.sg/Restaurant_Revi...,POINT (103.844475 1.32058)
...,...,...,...,...,...,...,...,...
11156,kml_79,POINT Z (103.84483401421 1.27640850068696 0),TANJONG PAGAR MRT STATION,EW15,GREEN,166.365574,https://www.tripadvisor.com.sg/Restaurant_Revi...,POINT (103.84376 1.277455)
11157,kml_221,POINT Z (103.889307215721 1.30881418793204 0),DAKOTA MRT STATION,CC8,YELLOW,1548.325966,https://www.tripadvisor.com.sg/Restaurant_Revi...,POINT (103.90249 1.304339)
11158,kml_330,POINT Z (103.938196740087 1.34563058756899 0),TAMPINES WEST MRT STATION,DT31,BLUE,174.913461,https://www.tripadvisor.com.sg/Restaurant_Revi...,POINT (103.938 1.3472)
11159,kml_123,POINT Z (103.856098851954 1.31188350800742 0),FARRER PARK MRT STATION,NE8,PURPLE,245.721596,https://www.tripadvisor.com.sg/Restaurant_Revi...,POINT (103.85711 1.313859)


## 4. Finding Walking Distance and Duration of Train Exit from Restaurant

#### Reading in Nearest Restaurant Exit Dataset

In [3]:
df_restaurant_nearest_exit = pd.read_csv('data/restaurant_nearest_station_exit.csv')

In [7]:
def extract_longitude(x):
    lon_lat_list = re.findall('\d+\.\d+', x)
    if len(lon_lat_list) > 1:
        return float(lon_lat_list[0])
    return np.nan

def extract_latitude(x):
    lon_lat_list = re.findall('\d+\.\d+', x)
    if len(lon_lat_list) > 1:
        return float(lon_lat_list[1])
    return np.nan

# geometry columns read in as string by read_csv
df_restaurant_nearest_exit['exit_lon'] = df_restaurant_nearest_exit['exit_geometry'].apply(extract_longitude)
df_restaurant_nearest_exit['exit_lat'] = df_restaurant_nearest_exit['exit_geometry'].apply(extract_latitude)
df_restaurant_nearest_exit['restaurant_lon'] = df_restaurant_nearest_exit['restaurant_geometry'].apply(extract_longitude)
df_restaurant_nearest_exit['restaurant_lat'] = df_restaurant_nearest_exit['restaurant_geometry'].apply(extract_latitude)

#### Use OpenRouteServices to extract walking distance and duration

In [ ]:
# ORS API token
from tokens import ors_token

ors_token = '############### USE OWN ORS TOKEN ###############'

In [5]:
# -------- Function to use Openrouteservice API to obtain walking distance and duration --- #
def find_travelling_route(start_pt_lon, start_pt_lat, end_pt_lon, end_pt_lat, ors_token):
    client = openrouteservice.Client(key=ors_token)
    coord = ((start_pt_lon, start_pt_lat), (end_pt_lon, end_pt_lat))
    route = client.directions(coord, profile='foot-walking')
    return route

In [185]:
# Note that ORS has a daily limit of 2000, so only 2000 routes can be retrieved
for idx, row in df_restaurant_nearest_exit.iterrows():
    x_lon = row['exit_lon']
    x_lat = row['exit_lat']
    y_lon = row['restaurant_lon']
    y_lat = row['restaurant_lat']

    # empty latitude or longitude
    if pd.isna(y_lon) or pd.isna(y_lat) or pd.isna(x_lat) or pd.isna(x_lon):
        continue

    try:
        route = find_travelling_route(x_lon, x_lat, y_lon, y_lat, ors_token)
        if 'distance' in route['routes'][0]['summary']:
            df_restaurant_nearest_exit.loc[idx, 'walking_distance'] = route['routes'][0]['summary']['distance']
            df_restaurant_nearest_exit.loc[idx, 'walking_duration'] = route['routes'][0]['summary']['duration']
    except:
        df_restaurant_nearest_exit.loc[idx, 'walking_distance'] = np.nan
        df_restaurant_nearest_exit.loc[idx, 'walking_duration'] = np.nan

In [214]:
df_restaurant_nearest_exit

,url,walking_distance,walking_duration
0,https://www.tripadvisor.com.sg/Restaurant_Revi...,269.4,193.9
1,https://www.tripadvisor.com.sg/Restaurant_Revi...,470.8,339.0
2,https://www.tripadvisor.com.sg/Restaurant_Revi...,580.0,417.6
3,https://www.tripadvisor.com.sg/Restaurant_Revi...,791.9,570.2
4,https://www.tripadvisor.com.sg/Restaurant_Revi...,83.1,59.8
...,...,...,...
11156,https://www.tripadvisor.com.sg/Restaurant_Revi...,222.3,160.1
11157,https://www.tripadvisor.com.sg/Restaurant_Revi...,1916.6,1380.0
11158,https://www.tripadvisor.com.sg/Restaurant_Revi...,212.5,153.0
11159,https://www.tripadvisor.com.sg/Restaurant_Revi...,354.4,255.2
